In [79]:
import requests
from unidecode import unidecode

draftkings_nfl_api_url = "https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/88808?format=json"
draftkings_nfl_site_url = "https://sportsbook/https:.draftkings.com/leagues/football/nfl"
draftkings_cbb_api_url = "https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/92483?format=json"

def get_dk_api_url(num):
    return f"https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/{num}?format=json"

baseball_id = 84240


In [80]:
# Code to help find the api - do not run
def find_access_code_in_server():
    response = requests.get(draftkings_nfl_site_url, headers={"User-Agent": "Mozilla/5.0"})
    if ("88808" in response.text):
        print("Found the event group id on the site")
        index_found = response.text.find("88808")
        print(response.text[index_found-500:index_found+500])
    else:
        print("Did not find the event group id on the site")
        print(response.text)

In [81]:
# put a request into the url
response = requests.get(get_dk_api_url(baseball_id), headers={"User-Agent": "Mozilla/5.0"})

json = response.json()
json['eventGroup'].keys()

dict_keys(['eventGroupId', 'displayGroupId', 'name', 'offerCategories', 'events', 'tags', 'nameIdentifier'])

In [82]:
category_urls = {}
cat_names = {}
for event in json['eventGroup']['offerCategories']:
    print(event['offerCategoryId'], event['name'])
    category_urls[event['offerCategoryId']] = "https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/" + str(baseball_id) + "/categories/" + str(event['offerCategoryId']) + "?format=json"
    cat_names[event['offerCategoryId']] = event['name']

print(category_urls)

493 Game Lines
988 Plate Appearance
735 Daily Specials
1024 1st Inning
729 Innings
743 Batter Props
1031 Pitcher Props
724 Game Props
1133 Correct Score
754 Game Parlays
1576 Series Winner
517 Team Futures
1568 Start of Season
684 Awards
1279 Player Totals
685 Regular Season Leaders
1297 League Seeding
972 Season Specials
{493: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/493?format=json', 988: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/988?format=json', 735: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/735?format=json', 1024: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/1024?format=json', 729: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/729?format=json', 743: 'https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/743?format=json', 1031: 'https://sportsbook.draft

In [83]:
url = "https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/{baseball_id}/categories/1218?format=json"
print(url)

response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
json = response.json()

https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/{baseball_id}/categories/1218?format=json


In [84]:
csv_data = []
queue = [cat_id for cat_id in category_urls]

while len(queue) > 0:
    cat_id = queue.pop(0)
    print(cat_id, "category!")

    response = requests.get(category_urls[cat_id], headers={"User-Agent": "Mozilla/5.0"})
    print(response.status_code, category_urls[cat_id])
    json = response.json()

    for category in json['eventGroup']['offerCategories']:
        if 'offerSubcategoryDescriptors' in category.keys():
            for discriptor in category['offerSubcategoryDescriptors']:
                if 'offerSubcategory' in discriptor.keys():
                    for offer in discriptor['offerSubcategory']['offers']:
                        for i in offer:
                            for outcome in i['outcomes']:
                                dict = outcome
                                dict['category'] = cat_names[cat_id]
                                if 'label' in i.keys():
                                    dict['prop_label'] = i['label']
                                else:
                                    print(i, cat_names[cat_id])
                                    continue
                                csv_data.append(dict)
                elif discriptor['subcategoryId'] not in category_urls.keys() and "subcategories" not in category_urls[cat_id]:
                    print(discriptor['name'], discriptor['subcategoryId'], cat_id)
                    queue.append(discriptor['subcategoryId'])
                    category_urls[discriptor['subcategoryId']] = "https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/" + str(baseball_id) + "/categories/" + str(cat_id) + "/subcategories/" + str(discriptor['subcategoryId']) + "?format=json"
                    cat_names[discriptor['subcategoryId']] = discriptor['name']



493 category!
200 https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/493?format=json
Listed Pitcher 6618 493
Alternate Run Line 13168 493
Alternate Total Runs 13169 493
Total Runs (3-Way) 6694 493
988 category!
200 https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/988?format=json
735 category!
200 https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/735?format=json
Highest Scoring Game 11065 735
Highest Scoring Team 6624 735
Any Game Over X+ Runs 11060 735
Any Team Over X+ Runs 11059 735
No Hitter 11511 735
1024 category!
200 https://sportsbook.draftkings.com//sites/US-SB/api/v5/eventgroups/84240/categories/1024?format=json
Hits 11214 1024
Scoring 12858 1024
Strikeouts 12855 1024
Pitches Thrown 12857 1024
Batters to the Plate 12856 1024
1st Inning Result 13740 1024
1st Inning Run Line 13736 1024
1st Inning Odd/Even 12862 1024
729 category!
200 https://sportsbook.draftkings.com//sites/US-SB/ap

In [85]:
# unidecode everything in csv_data
for i in csv_data:
    for key in i:
        if type(i[key]) == str:
            i[key] = unidecode(i[key])

In [86]:
# turn the data in csv_data into a csv file
import csv

with open('draftkings_mlb.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_data[0].keys())
    for row in csv_data:
        writer.writerow(row.values())

In [87]:
csv_data[0]

{'providerOutcomeId': '0HC74554603P150_3',
 'providerId': 2,
 'providerOfferId': '2_74554603',
 'label': 'LA Angels',
 'oddsAmerican': '-130',
 'oddsDecimal': 1.76923077,
 'oddsDecimalDisplay': '1.76',
 'oddsFractional': '10/13',
 'line': 1.5,
 'participant': 'LA Angels',
 'participantType': 'Team',
 'main': True,
 'sortOrder': 1499,
 'tags': ['MainPointLine'],
 'participants': [{'id': '29901', 'name': 'LA Angels', 'type': 'Team'}],
 'category': 'Game Lines',
 'prop_label': 'Run Line'}